## Lab 1: 패션/뷰티 이커머스 에이전트 프로토타입 생성

### 개요

[Amazon Bedrock AgentCore](https://aws.amazon.com/bedrock/agentcore/)를 활용하여 한국 패션/뷰티 온라인 쇼핑몰 전용 고객 지원 에이전트를 구축합니다. 반품/교환에 특화된 서비스를 제공합니다.

**워크숍 여정:**
- **Lab 1 (현재)**: 에이전트 프로토타입 - 반품/교환 특화 에이전트 구축
- **Lab 2**: 메모리 강화 - 고객 선호도 및 구매 이력 추가
- **Lab 3**: Gateway & Identity - 반품 자격 검증 API 연동
- **Lab 4**: 프로덕션 배포 - AgentCore Runtime으로 관측성 확보
- **Lab 5**: 사용자 인터페이스 - 한국어 고객 인터페이스 구축

이 첫 번째 랩에서는 패션/뷰티 전문 고객 지원 에이전트 프로토타입을 구축합니다. 우리의 에이전트는 다음과 같은 전용 도구를 가집니다:
- **process_return()** - 반품 신청 및 처리 (사이즈/색상/품질/변심)
- **process_exchange()** - 교환 신청 및 처리 (빠른 교환 서비스)
- **web_search()** - 스타일링 팁, 사용법, 트렌드 정보 검색

### Lab 1을 위한 아키텍처
<div style="text-align:left">
    <img src="images/architecture_lab1_strands.png" width="75%"/>
</div>

*로컬에서 실행되는 간단한 프로토타입입니다. 이후 랩에서 AgentCore 서비스를 통해 공유 도구, 영구 메모리, 프로덕션급 관측성을 갖춘 시스템으로 마이그레이션할 예정입니다.*

### 전제 조건

* **AWS 계정** 및 적절한 권한
* **Python 3.10+** 로컬 설치
* **AWS CLI 구성** 및 자격 증명
* **Anthropic Claude 3.7** [Amazon Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html)에서 활성화
* **Strands Agents** 및 기타 라이브러리는 다음 셀에서 설치

### Step 1: 의존성 설치 및 라이브러리 가져오기
시작하기 전에 이 랩에 필요한 전제 조건을 설치해보겠습니다

In [ ]:
# 필요한 패키지 설치 (uv 사용)
# !uv pip install -e .

이제 필요한 라이브러리를 가져오고 boto3 세션을 초기화할 수 있습니다

In [ ]:
# 라이브러리 가져오기
import boto3
from boto3.session import Session

from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

from strands.tools import tool
from datetime import datetime
import random

In [ ]:
# boto 세션 가져오기
boto_session = Session()
region = boto_session.region_name
print(f"사용 중인 AWS 리전: {region}")

### Step 2: 맞춤 도구 구현

다음으로, 패션/뷰티 고객 지원 에이전트에 제공될 3가지 도구를 구현하겠습니다.

Strands Agent에서 도구를 정의하는 것은 매우 간단합니다. 함수에 `@tool` 데코레이터를 추가하고 함수의 docstring에서 도구에 대한 설명을 제공하기만 하면 됩니다. Strands Agents는 함수 문서, 타이핑 및 인수를 사용하여 에이전트에게 이 도구에 대한 컨텍스트를 제공합니다.

#### 도구 1: 반품 처리

**목적:** 패션/뷰티 제품의 반품 신청을 처리합니다. 사이즈 불일치, 색상 차이, 품질 문제, 단순 변심 등 다양한 사유에 대해 자동화된 처리를 제공합니다.

In [ ]:
@tool
def process_return(order_number: str, item_name: str, reason: str) -> str:
    """
    반품 신청을 처리합니다. 패션/뷰티 제품 특화.
    
    Args:
        order_number: 주문번호 (예: 'KS-2024-001234')
        item_name: 반품할 상품명
        reason: 반품 사유 ('사이즈', '색상', '품질', '변심' 등)
    
    Returns:
        반품 처리 결과 및 다음 단계 안내
    """
    
    # 패션/뷰티 반품 정책
    return_policies = {
        "패션": {
            "period": "7일",
            "conditions": [
                "택(tag) 제거하지 않았을 것",
                "착용 흔적이나 세탁 흔적이 없을 것", 
                "원래 포장 상태 유지",
                "향수나 화장품 냄새가 배지 않았을 것"
            ],
            "auto_approve": ["사이즈 불일치", "색상 차이", "품질 불량", "오배송"],
            "shipping_fee": {
                "사이즈": "무료 (판매자 부담)",
                "색상": "무료 (판매자 부담)", 
                "품질": "무료 (판매자 부담)",
                "변심": "3,000원 (고객 부담)"
            }
        },
        "뷰티": {
            "period": "7일",
            "conditions": [
                "미개봉 상태일 것",
                "봉인 스티커가 훼손되지 않았을 것",
                "사용하지 않았을 것"
            ],
            "auto_approve": ["알레르기", "색상 차이", "품질 불량", "오배송"],
            "shipping_fee": {
                "알레르기": "무료 (판매자 부담)",
                "색상": "무료 (판매자 부담)",
                "품질": "무료 (판매자 부담)", 
                "변심": "3,000원 (고객 부담)"
            }
        }
    }
    
    # 카테고리 자동 판별
    category = "뷰티" if any(keyword in item_name.lower() for keyword in 
                           ["립스틱", "파운데이션", "아이섀도", "스킨", "크림", "세럼", "향수"]) else "패션"
    
    policy = return_policies[category]
    
    # 자동 승인 여부 판단
    is_auto_approved = any(auto_reason in reason for auto_reason in policy["auto_approve"])
    
    # 접수 번호 생성
    return_id = f"RT-{datetime.now().strftime('%Y%m%d')}-{random.randint(1000, 9999)}"
    
    # 배송비 계산
    shipping_fee = "무료"
    for fee_reason, fee in policy["shipping_fee"].items():
        if fee_reason in reason:
            shipping_fee = fee
            break
    
    result = f"""✅ 반품 신청이 접수되었습니다.

📋 접수 정보
━━━━━━━━━━━━━━━━━━━━━━━━━━
• 접수번호: {return_id}
• 주문번호: {order_number} 
• 상품명: {item_name}
• 반품사유: {reason}
• 상품 카테고리: {category}
• 접수일시: {datetime.now().strftime('%Y-%m-%d %H:%M')}

"""
    
    if is_auto_approved:
        result += f"""✅ 반품 승인 완료 (자동 승인)
━━━━━━━━━━━━━━━━━━━━━━━━━━
🎯 다음 단계:
1. 📧 반품 접수 확인 문자가 발송됩니다
2. 📦 상품을 원래 포장재에 넣어 준비해 주세요
3. 🚚 택배기사님이 내일 오전 방문 예정입니다
4. ✅ 회수 완료 후 1-2일 내 환불 처리

💰 환불 정보:
• 배송비: {shipping_fee}
• 환불 예상일: 회수 후 1-2 영업일
• 환불 방법: 원 결제수단으로 자동 환불"""
        
        # 고객 만족도 향상 메시지
        if reason in ["사이즈 불일치", "사이즈"]:
            result += f"""

💡 다음 구매 시 도움말:
• 상품 상세페이지의 '실측 사이즈'를 확인해 주세요
• 평소 착용하시는 옷의 실측을 비교해 보세요
• 브랜드별로 사이즈가 다를 수 있습니다
• 궁금하시면 언제든 상담 문의해 주세요!"""
    else:
        result += f"""⏳ 반품 신청 검토 중
━━━━━━━━━━━━━━━━━━━━━━━━━━
🔍 검토 절차:
1. 담당자가 반품 사유를 검토합니다 (4시간 이내)
2. 승인 시 회수 일정 안내 문자 발송
3. 상품 회수 및 검수 진행
4. 최종 승인 후 환불 처리

💰 예상 비용:
• 배송비: {shipping_fee}
• 처리기간: 최대 3-5 영업일"""
    
    result += f"""

📞 문의사항:
• 고객센터: 1588-0000 (평일 9-18시)
• 카카오톡: @kstyle (24시간 상담)
• 접수번호로 진행상황 실시간 조회 가능

감사합니다. 더 나은 쇼핑 경험을 위해 노력하겠습니다! 💝"""
    
    return result

print("✅ 반품 처리 도구 준비 완료")

#### 도구 2: 교환 처리

**목적:** 빠른 교환 서비스를 제공합니다. 사이즈, 색상 교환뿐만 아니라 동일 가격대의 다른 상품으로의 교환도 지원합니다.

In [ ]:
@tool
def process_exchange(order_number: str, item_name: str, current_option: str, desired_option: str) -> str:
    """
    교환 신청을 처리합니다. 빠른 교환 서비스 제공.
    
    Args:
        order_number: 주문번호
        item_name: 교환할 상품명
        current_option: 현재 옵션 (예: "화이트/M")
        desired_option: 원하는 옵션 (예: "블랙/L")
    
    Returns:
        교환 처리 결과 및 재고 확인
    """
    
    # 교환 접수번호 생성
    exchange_id = f"EX-{datetime.now().strftime('%Y%m%d')}-{random.randint(1000, 9999)}"
    
    # 재고 시뮬레이션
    stock_status = random.choice(["재고 있음", "재고 부족 (2-3일 소요)", "일시 품절"])
    
    result = f"""✅ 교환 신청이 접수되었습니다.

📋 교환 정보
━━━━━━━━━━━━━━━━━━━━━━━━━━
• 접수번호: {exchange_id}
• 주문번호: {order_number}
• 상품명: {item_name}
• 현재 옵션: {current_option}
• 희망 옵션: {desired_option}
• 접수일시: {datetime.now().strftime('%Y-%m-%d %H:%M')}

"""
    
    if stock_status == "재고 있음":
        result += f"""✅ 교환 상품 재고 확인 완료
━━━━━━━━━━━━━━━━━━━━━━━━━━
🚀 K-Style 빠른 교환 서비스:

📦 교환 프로세스:
1. 오늘 오후 5시까지 기존 상품 회수
2. 동시에 새 상품 배송 출발  
3. 내일 오전 중 새 상품 도착 예정
4. 회수와 배송이 동시에 진행됩니다!

💝 특별 혜택:
• 교환 배송비: 완전 무료
• 당일 처리: 재고 있음
• 동시 교환: 기다림 없이 바로!"""
    
    elif "부족" in stock_status:
        result += f"""⏳ 교환 상품 재고 확인 중
━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 재고 상황: {stock_status}

📅 교환 일정:
1. 오늘 회수 진행
2. 새 상품 입고 후 즉시 발송 ({stock_status.split('(')[1].split(')')[0]})
3. 회수 완료 시 임시 쿠폰 발급

🎁 기다려주셔서 감사합니다:
• 10% 할인 쿠폰 발급 (다음 구매 시)
• 무료 배송 + 포장 업그레이드"""
        
    else:  # 품절
        result += f"""😔 교환 상품 일시 품절
━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 재고 상황: {stock_status}

🔄 대안 제안:
1. 💰 전액 환불 (1-2일 내 처리)
2. 🎯 유사 상품 추천 (같은 가격대)
3. 📅 재입고 알림 신청 (우선 주문권 제공)
4. 🏷️ 다른 색상/사이즈 확인

🎁 불편을 드려 죄송합니다:
• 15% 할인 쿠폰 발급
• 다음 주문 시 무료배송 + 무료 포장"""
    
    result += f"""

🔍 교환 가능 조건:
• 택(tag) 제거하지 않았을 것
• 착용이나 사용 흔적이 없을 것
• 세탁하지 않았을 것
• 원래 포장 상태 유지

📞 추가 문의:
• 고객센터: 1588-0000
• 카카오톡: @kstyle
• 교환 진행상황 실시간 알림 제공

더 완벽한 스타일을 위해 최선을 다하겠습니다! ✨"""
    
    return result

print("✅ 교환 처리 도구 준비 완료")

#### 도구 3: 웹 검색 (패션/뷰티 특화)

**목적:** 스타일링 조언, 사용법, 트렌드 정보를 제공하여 고객의 쇼핑 경험을 향상시킵니다.

In [ ]:
@tool
def web_search(keywords: str, region: str = "kr-kr", max_results: int = 5) -> str:
    """웹에서 최신 정보를 검색합니다.
    
    Args:
        keywords (str): 검색 쿼리 키워드
        region (str): 검색 지역: wt-wt, us-en, uk-en, kr-kr, ru-ru 등
        max_results (int | None): 반환할 최대 결과 수
    Returns:
        검색 결과를 포함한 딕셔너리 리스트
    
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        print("web_search results:\n", results)
        return results if results else "결과를 찾을 수 없습니다."
    except RatelimitException:
        return "요청 한도에 도달했습니다. 나중에 다시 시도해 주세요."
    except DDGSException as e:
        return f"검색 오류: {e}"
    except Exception as e:
        return f"검색 오류: {str(e)}"

print("✅ 웹 검색 도구 준비 완료")

### Step 4: 패션/뷰티 고객 지원 에이전트 생성 및 구성

다음으로, 모델, 이전 단계에서 구현된 도구 목록, 그리고 시스템 프롬프트를 제공하여 패션/뷰티 고객 지원 에이전트를 생성하겠습니다.

In [ ]:
from strands import Agent
from strands.models import BedrockModel

# 패션/뷰티 전문 고객 지원 시스템 프롬프트
SYSTEM_PROMPT = """당신은 한국 패션/뷰티 전문 온라인 쇼핑몰 'K-Style'의 친절하고 전문적인 고객 상담원입니다.

🏪 K-Style 쇼핑몰 소개:
- 패션: 여성/남성 의류, 신발, 가방, 액세서리
- 뷰티: 스킨케어, 메이크업, 향수, 헤어케어
- 전문 서비스: 반품/교환 당일 처리, 스타일링 상담

👨‍💼 상담원 역할:
- 반품/교환 신청을 신속하고 정확하게 처리
- 사이즈, 색상, 스타일 관련 전문 상담 제공
- 패션 트렌드 및 뷰티 사용법 안내
- 항상 존댓말 사용, 친근하고 전문적인 응대
- 고객의 스타일과 선호도를 고려한 맞춤 서비스

🛠️ 사용 가능한 도구:
1. process_return() - 반품 신청 및 처리 (사이즈/색상/품질/변심)
2. process_exchange() - 교환 신청 및 처리 (빠른 교환 서비스)
3. web_search() - 스타일링 팁, 사용법, 트렌드 정보 검색

💡 응대 원칙:
- 반품/교환은 고객의 당연한 권리임을 인식
- 사이즈 가이드와 실제 착용감의 차이를 이해
- 온라인 쇼핑의 한계를 공감하며 최선의 해결책 제시
- 재구매 의향을 높이는 긍정적 경험 제공"""

# Bedrock 모델 초기화 (Anthropic Claude 3.7 Sonnet)
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    temperature=0.3,
    region_name=region
)

# 모든 도구를 갖춘 패션/뷰티 고객 지원 에이전트 생성
agent = Agent(
    model=model,
    tools=[
        process_return,     # 도구 1: 반품 처리
        process_exchange,   # 도구 2: 교환 처리
        web_search         # 도구 3: 패션/뷰티 정보 검색
    ],
    system_prompt=SYSTEM_PROMPT,
)

print("✅ 패션/뷰티 고객 지원 에이전트가 성공적으로 생성되었습니다!")

### Step 5: 패션/뷰티 고객 지원 에이전트 테스트

모든 기능이 올바르게 작동하는지 확인하기 위해 샘플 쿼리로 에이전트를 테스트해보겠습니다.

#### 테스트 1: 반품 처리

In [ ]:
# 반품 문의 테스트
response = agent("지난주 산 원피스 사이즈가 작아요. 반품하고 싶어요. 주문번호는 KS-2024-001234이고 상품명은 '플라워 패턴 원피스'예요.")
print(response)

#### 테스트 2: 교환 처리

In [ ]:
# 교환 문의 테스트
response = agent("이 청바지를 M에서 L로 교환할 수 있나요? 주문번호 KS-2024-001235, 현재 '네이비/M'인데 '네이비/L'로 바꾸고 싶어요.")
print(response)

#### 테스트 3: 스타일링 조언

In [ ]:
# 스타일링 조언 테스트
response = agent("플리츠 스커트에 어떤 상의가 어울려요? 코디 방법을 알려주세요.")
print(response)

#### 테스트 4: 뷰티 사용법

In [ ]:
# 뷰티 사용법 테스트
response = agent("쿠션 파운데이션 올바른 사용법을 알려주세요. 처음 써보는데 어떻게 발라야 하나요?")
print(response)

## 🎉 Lab 1 완료!

패션/뷰티 전문 고객 지원 에이전트 프로토타입을 성공적으로 구축했습니다! 달성한 내용은 다음과 같습니다:

- 반품/교환에 특화된 3가지 맞춤 도구 구축
- 패션/뷰티 도메인 전문 지식 통합
- 한국 이커머스 문화에 맞는 고객 응대 방식 구현
- 프로덕션 여정을 위한 기반 구축

### 현재 한계점 (다음 랩에서 해결 예정!)
- **단일 사용자 대화 메모리** - 로컬 대화 세션, 여러 고객을 위해서는 여러 세션 필요
- **세션 제한 대화 이력** - 장기 메모리나 세션 간 정보 부재
- **도구 재사용성** - 다른 에이전트 간 도구 공유 불가
- **로컬 실행만** - 확장 불가능
- **Identity 부재** - 사용자/에이전트 식별이나 접근 제어 없음
- **관측성 제한** - 에이전트 행동에 대한 제한적 관측성
- **기존 API 부재** - 고객 데이터를 위한 기존 엔터프라이즈 API 액세스 없음

### 특별한 성과
- **패션/뷰티 특화**: 업계별 용어와 프로세스 반영
- **한국 문화 적응**: 존댓말, 고객 서비스 문화 고려
- **실용적 조언**: 실제 도움이 되는 스타일링/사용법 제공
- **감정적 배려**: 반품/교환 시 고객 심리 고려

##### 다음 단계 [Lab 2: 고객 선호도 메모리로 에이전트 개인화 →](lab-02-agentcore-memory.ipynb)